In [1]:
#iris 데이터셋 로드
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)
print(X[:5])
print(y[:5])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
[0 0 0 0 0]


In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10, test_size=0.2, stratify=y, shuffle=True)
print(len(X_train))

120


In [3]:
#신경망 모형
from torch import nn
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50) # (input nodes, output nodes:50)
        self.layer2 = nn.Linear(50, 20)
        self.layer3 = nn.Linear(20, 3)

    # 활성화함수
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        # 출력층의 활성화함수 - 소프트맥스
        x = F.softmax(self.layer3(x), dim=0)
        return x

In [4]:
import torch
# 초기화함수의 input_dim
model = Model(X_train.shape[1])
# 최적화함수
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# 손실함수
loss_fn = nn.CrossEntropyLoss()
epochs = 100

In [5]:
from torch.autograd import Variable
import torch.nn.functional as F
# 넘파이배열을 텐서로 변환
X_train = Variable(torch.from_numpy(X_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()
for epoch in range(1, epochs + 1):
    print('Epoch', epoch)
    # 예측값
    y_pred = model(X_train)
    
    # 손실함수
    loss = loss_fn(y_pred, y_train)
    print('loss:', loss.item())

    # 경사 초기화
    optimizer.zero_grad()
    loss.backward() # 역전파
    optimizer.step() # 가중치 업데이트

Epoch 1
loss: 1.0989898443222046
Epoch 2
loss: 1.0982489585876465
Epoch 3
loss: 1.0977082252502441
Epoch 4
loss: 1.097055435180664
Epoch 5
loss: 1.096272587776184
Epoch 6
loss: 1.095436692237854
Epoch 7
loss: 1.0945677757263184
Epoch 8
loss: 1.0936888456344604
Epoch 9
loss: 1.092803716659546
Epoch 10
loss: 1.0919359922409058
Epoch 11
loss: 1.0912119150161743
Epoch 12
loss: 1.0906003713607788
Epoch 13
loss: 1.0900397300720215
Epoch 14
loss: 1.0895307064056396
Epoch 15
loss: 1.089080810546875
Epoch 16
loss: 1.0886743068695068
Epoch 17
loss: 1.088299036026001
Epoch 18
loss: 1.087955117225647
Epoch 19
loss: 1.087653636932373
Epoch 20
loss: 1.0874038934707642
Epoch 21
loss: 1.0871952772140503
Epoch 22
loss: 1.087029218673706
Epoch 23
loss: 1.0868757963180542
Epoch 24
loss: 1.0866955518722534
Epoch 25
loss: 1.0865217447280884
Epoch 26
loss: 1.0863895416259766
Epoch 27
loss: 1.0862674713134766
Epoch 28
loss: 1.086138367652893
Epoch 29
loss: 1.086004614830017
Epoch 30
loss: 1.0858525037765503


In [6]:
#예측값
X_test = Variable(torch.from_numpy(X_test)).float()
pred = model(X_test)
pred[:5]

tensor([[2.6995e-06, 8.1283e-03, 2.2304e-04],
        [3.9232e-02, 3.0258e-06, 5.3297e-11],
        [1.1415e-06, 2.9226e-03, 5.5198e-04],
        [2.2819e-07, 4.6139e-04, 2.7914e-03],
        [1.3576e-04, 3.3906e-01, 3.7157e-06]], grad_fn=<SliceBackward0>)

In [7]:
import numpy as np
np.argmax(pred.data.numpy(), axis=1)

array([1, 0, 1, 2, 1, 2, 0, 2, 2, 0, 0, 1, 2, 2, 1, 0, 0, 1, 2, 0, 2, 2,
       2, 0, 0, 1, 1, 0, 1, 1], dtype=int64)

In [9]:
#모형의 정확도 측정
from sklearn.metrics import accuracy_score
accuracy_score(y_test, np.argmax(pred.data.numpy(), axis=1))

1.0

In [10]:
# 모델 저장
torch.save(model, 'c:/data/iris/iris_torch.model')

In [11]:
# 모델 로드
model2 = torch.load('c:/data/iris/iris_torch.model')
np.argmax(model2(X_test[0]).data.numpy())

2

In [12]:
from torchinfo import summary
summary(model, input_size=(32,4))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [32, 3]                   --
├─Linear: 1-1                            [32, 50]                  250
├─Linear: 1-2                            [32, 20]                  1,020
├─Linear: 1-3                            [32, 3]                   63
Total params: 1,333
Trainable params: 1,333
Non-trainable params: 0
Total mult-adds (M): 0.04
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.01
Estimated Total Size (MB): 0.02